In [6]:
import pymysql
import pandas as pd

In [7]:
def query_SQL():
    # Get fields from business table that need to be used to populate reviews table (and also are reproduced in reviews table for convenience)
    con = False
    rows = []
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='ortho')
        with con:
            cur = con.cursor()
            sql = 'USE ortho;'
            cur.execute(sql)
            #sql = 'SELECT id,term_id,city_id FROM business WHERE yelp_id = "'+yelp_business_id+'";' 
            sql = 'SELECT id, yelp_id, term_id, city_id FROM business;' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [8]:
def make_bus_df(rows):
    # receives as argument, the output of the MySQL query of ortho.business table
    
    # UNPACK ROWS - no matter which order you place these indices[][], if one is a : it will give you everything in the row indexed by the number you gave 
    bid=[];yid=[];tid=[];cid=[];
    for row in rows:
        #print row[0],row[1],row[2]
        bid.append(row[0])
        yid.append(row[1])
        tid.append(row[2])
        cid.append(row[3])
    #print(bid[:5],yid[:5])
    
    # Make Pandas DataFrame:  business-level ids
    bus_df = pd.DataFrame([bid,yid,tid,cid]).transpose()
    bus_df.columns=['bid','yelp_id','term_id','city_id']
    #print '\n',bus_df.head(5)
    return bus_df

In [43]:
def main():
    rows = query_SQL()#response is shape rev# x 4
    business_df = make_bus_df(rows)
    
    #print business_df.head(5),'\n'
    
    dups = business_df.duplicated(subset="yelp_id")
    
    count=0
    dup_idx = []
    for dup in dups:
        if dup==True:
            #print count
            dup_idx.append(count)
        count+=1
    #print dup_idx
    
    print business_df.ix[dup_idx,['bid','yelp_id']]

    
if __name__=="__main__":
    main()

     bid                                       yelp_id
200  503     peninsula-orthopedic-associates-daly-city
207  510                     khan-a-shabi-md-daly-city
217  520                  kenneth-hunt-md-redwood-city
218  521                  allan-k-mishra-md-menlo-park
219  522                jason-l-dragoo-md-redwood-city
224  527                wall-michael-s-md-redwood-city
225  528  stanford-sports-medicine-clinic-redwood-city
233  536                bradley-p-graw-md-redwood-city
234  537                 gary-s-fanton-md-redwood-city
238  541                robert-millard-md-redwood-city
248  551                paul-eliot-hughes-md-san-mateo
253  556                 marc-r-safran-md-redwood-city
257  560     stanford-orthopaedic-surgery-redwood-city
278  581               hazelrig-paul-e-jr-md-san-mateo
286  589                   don-young-park-md-san-mateo
288  591  san-mateo-orthopedic-medical-group-san-mateo
289  592             liza-reichenberger-pa-c-san-mateo
290  593  